Automatic parameter estimation for GR4J. Uses scipy optimize packages.

Following exemples found in https://stackoverflow.com/questions/19664865/migrating-from-pulp-to-scipy

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.optimize as opt

# GR4J lives in a directory above this code. Not sure this is the best practice...
import os, sys
sys.path.append(os.path.relpath("../gr4j"))
from gr4j import gr4j

# Reading USGS data from GR4J packaage test dirlendo o arquivo de dados do USGS
test_data = pd.read_csv('../gr4j/tests/USGS_02430680_combined.csv', skiprows=1)

# Initial guess of model parameters
# done as a list since it's what the optimization function expects.
# GR4J funcion recieves a dictionary
# This is treated inside the funcion to be optmized. 
#guess = [303.627616, 0.32238919, 6.49759466, 0.294803885]
guess = [103.627616, 0.32238919, 6.49759466, 0.294803885]

# Objective function (Nash-Sutcliffe)
def nash(qobs, qsim):
    top = sum((qsim - qobs)**2)
    bottom = sum((qobs.mean() - qobs)**2)
    return 1 - top/bottom

# function to be minimized
def fun(guess):
    params = {'X1': guess[0],
              'X2': guess[1],
              'X3': guess[2],
              'X4': guess[3]
        }
    states = {'production_store': 0.60 * params['X1'],
          'routing_store': 0.70 * params['X3']
         }
    prec = test_data['P'].ix[:700]
    etp = test_data['PE'].ix[:700]
    qobs = test_data['Q'].ix[:700]
    qsim = gr4j(prec, etp, params, states)
    n = nash(qobs, qsim)
    return -n


From here on things are still **broken**. Simulated discharge is in mm/day (I believe, from looking at the model excell spreadsheet). But observed discharge from the USGS dataset is in cubic meters per day. Some conversions must be done in the test data prior to running the optimization.

Also, it's interesnting to make the function to be optimized more generic, taking as input not only the parameters to be estimated but precipitation, etp and qobs.

In [5]:
result = opt.minimize(fun, guess, method='SLSQP')

# Simulating using the optimized parameters

params = {'X1': result['x'][0],
          'X2': result['x'][1],
          'X3': result['x'][2],
          'X4': result['x'][3]
         }

states = {'production_store': 0.60 * params['X1'],
          'routing_store': 0.70 * params['X3']
         }

prec = test_data['P'].loc[:700]
etp = test_data['PE'].loc[:700]
qobs = test_data['Q'].loc[:700]
qsim = gr4j(prec, etp, params, states)

# qobs is in cubic meters per seccond
# GR4J gives results in mm/day (at least thats that I understood from the model excell spreadsheet)
# must do some actual conversions in the observed data prior to optimization
nash(qobs, qsim)

-0.096392218390526585